In [1]:
import findspark
findspark.init('/home/sdmohant/spark-3.0.0-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark import SparkContext,SparkConf

In [3]:
from pyspark.sql import SparkSession

In [4]:
#spark = SparkSession.builder.appName("MyApp").getOrCreate()
#spark = SparkSession.builder.appName("MyName").getOrCreate()
conf = SparkConf().set("spark.cores.max", "16") \
    .set("spark.driver.memory", "16g") \
    .set("spark.executor.memory", "16g") \
    .set("spark.executor.memory_overhead", "16g") \
    .set("spark.driver.maxResultsSize", "4g") \
    .set("spark.sql.shuffle.partitions","300")

sc = SparkContext(appName="newApp", conf=conf)
spark = SparkSession(sc)

/home/sdmohant/spark-3.0.0-bin-hadoop2.7/python/pyspark/context.py:220: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [5]:
from pyspark.sql.functions import isnan, count, col, when
from pyspark.sql.types import *

## FieldsOfStudy

In [6]:
FOS_DF = spark.read.options(header= True,inferSchema=True,delimiter=",").csv("FOS.csv")

In [7]:
FOS_DF.show(2)

+----------+-------+-----------------+--------------+---------------+-------------+----------------+
|    FOS_ID|FOSRank|FOSNormalizedName|FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+----------+-------+-----------------+--------------+---------------+-------------+----------------+
|2775844904|  14256|            enmap|         EnMAP|              3|          149|            1006|
|2775859947|  25000|     least darter|  Least darter|              3|            8|              40|
+----------+-------+-----------------+--------------+---------------+-------------+----------------+
only showing top 2 rows



In [8]:
FOS_zero = FOS_DF.filter("FOSDisplayLevel == 0").collect()

In [9]:
FOS_zero

[Row(FOS_ID=127313418, FOSRank=9480, FOSNormalizedName=u'geology', FOSDisplayName=u'Geology', FOSDisplayLevel=0, FOSPaperCount=3226880, FOSCitationCount=30483377),
 Row(FOS_ID=142362112, FOSRank=25000, FOSNormalizedName=u'art', FOSDisplayName=u'Art', FOSDisplayLevel=0, FOSPaperCount=4792512, FOSCitationCount=2391299),
 Row(FOS_ID=185592680, FOSRank=7560, FOSNormalizedName=u'chemistry', FOSDisplayName=u'Chemistry', FOSDisplayLevel=0, FOSPaperCount=14843182, FOSCitationCount=164570817),
 Row(FOS_ID=121332964, FOSRank=7739, FOSNormalizedName=u'physics', FOSDisplayName=u'Physics', FOSDisplayLevel=0, FOSPaperCount=7393700, FOSCitationCount=66954797),
 Row(FOS_ID=95457728, FOSRank=12996, FOSNormalizedName=u'history', FOSDisplayName=u'History', FOSDisplayLevel=0, FOSPaperCount=5847639, FOSCitationCount=4495040),
 Row(FOS_ID=144133560, FOSRank=25000, FOSNormalizedName=u'business', FOSDisplayName=u'Business', FOSDisplayLevel=0, FOSPaperCount=3362436, FOSCitationCount=10493865),
 Row(FOS_ID=2056

## FOS Children

In [10]:
ParentChild_DF = spark.read.options(header= True,inferSchema=True,delimiter=",").csv("ParentChildFOS.csv")

In [11]:
ParentChild_DF.head()

Row(FOS_ID=107038049, FOSChildID=503158894)

In [12]:
type(ParentChild_DF)

pyspark.sql.dataframe.DataFrame

## Geology 127313418

In [13]:
childOfRoot = ParentChild_DF.filter(ParentChild_DF.FOS_ID == 127313418).collect()##parent level

In [14]:
len(childOfRoot)

22

In [15]:
ParentChild_DF = ParentChild_DF.toPandas()

In [16]:
#Children at level 2
childAtLevel2 = []
for x in childOfRoot:
    for y in ParentChild_DF.index:
        #print(ChildFOS_df["FOS_ID"][y])
        #print("\n")
        #print(childOfRoot[x]["FOSChildID"])
        if ParentChild_DF["FOS_ID"][y] == x["FOSChildID"]:
            childAtLevel2.append(ParentChild_DF["FOSChildID"][y])
            


In [17]:
len(childAtLevel2)

190

In [18]:
#Children at level 3
childAtLevel3 = []
for x in childAtLevel2:
    #print(x)
    for y in ParentChild_DF.index:
        
       # print(type(ChildFOS_df["FOS_ID"][y]))
        if ParentChild_DF["FOS_ID"][y] == x:            
            childAtLevel3.append(ParentChild_DF["FOSChildID"][y])

In [19]:
len(childAtLevel3)

0

In [20]:
#geology_fos = list(childOfRoot["FOSChildID"]) + list(childAtLevel2)

In [21]:
childAtLevel3 = ParentChild_DF[ParentChild_DF["FOS_ID"] == 117467278]

In [22]:
childAtLevel3

,FOS_ID,FOSChildID


In [23]:
childOfRoot = ParentChild_DF[ParentChild_DF["FOS_ID"] == 127313418]

In [24]:
len(childOfRoot)

22

In [25]:
#Children at level 2
childAtLevel2 = set()
for x in childOfRoot.index:
    for y in ParentChild_DF.index:
        #print(ChildFOS_df["FOS_ID"][y])
        #print("\n")
        #print(x)
        if ParentChild_DF["FOS_ID"][y] == childOfRoot["FOSChildID"][x]:
            childAtLevel2.add(ParentChild_DF["FOSChildID"][y])
            

In [26]:
len(childAtLevel2)

148

In [27]:
#Children at level 3
childAtLevel3 = set()
for x in childAtLevel2:
    for y in ParentChild_DF.index:
        if ParentChild_DF["FOS_ID"][y] == x:
            childAtLevel3.add(ParentChild_DF["FOSChildID"][y])

In [28]:
geology_fos= list(childOfRoot["FOSChildID"]) + list(childAtLevel2)

In [29]:
len(geology_fos)

170

## Art 142362112

In [30]:
childOfRoot = ParentChild_DF[ParentChild_DF["FOS_ID"] == 142362112]

In [31]:
len(childOfRoot)

6

In [32]:
#Children at level 2
childAtLevel2 = set()
for x in childOfRoot.index:
    for y in ParentChild_DF.index:
        #print(ChildFOS_df["FOS_ID"][y])
        #print("\n")
        #print(x)
        if ParentChild_DF["FOS_ID"][y] == childOfRoot["FOSChildID"][x]:
            childAtLevel2.add(ParentChild_DF["FOSChildID"][y])
            

In [33]:
len(childAtLevel2)

112

In [34]:
#Children at level 3
childAtLevel3 = set()
for x in childAtLevel2:
    for y in ParentChild_DF.index:
        if ParentChild_DF["FOS_ID"][y] == x:
            childAtLevel3.add(ParentChild_DF["FOSChildID"][y])

In [35]:
len(childAtLevel3)

0

In [36]:
art_fos= list(childOfRoot["FOSChildID"]) + list(childAtLevel2)

In [37]:
len(art_fos)

118

## Chemistry 185592680

In [38]:
childOfRoot = ParentChild_DF[ParentChild_DF["FOS_ID"] == 185592680]

In [39]:
len(childOfRoot)

26

In [40]:
#Children at level 2
childAtLevel2 = set()
for x in childOfRoot.index:
    for y in ParentChild_DF.index:
        #print(ChildFOS_df["FOS_ID"][y])
        #print("\n")
        #print(x)
        if ParentChild_DF["FOS_ID"][y] == childOfRoot["FOSChildID"][x]:
            childAtLevel2.add(ParentChild_DF["FOSChildID"][y])
            

In [41]:
len(childAtLevel2)

110

In [42]:
#Children at level 3
childAtLevel3 = set()
for x in childAtLevel2:
    for y in ParentChild_DF.index:
        if ParentChild_DF["FOS_ID"][y] == x:
            childAtLevel3.add(ParentChild_DF["FOSChildID"][y])

In [43]:
len(childAtLevel3)

0

In [44]:
type(childOfRoot)

pandas.core.frame.DataFrame

In [45]:
childOfRoot["FOSChildID"]

458    179104552
459     31903555
460    115704247
461     55493867
462    159985019
463    159467904
464    188027245
465     71240020
466    147789679
467    183696295
468    199289684
469    178790620
470     43617362
471     13965031
472     21951064
473    126348684
474     41999313
475    191897082
476    171250308
477    107872376
478    177322064
479     75473681
480      8010536
481     46141821
482    155647269
483    147597530
Name: FOSChildID, dtype: int64

In [46]:
type(childAtLevel2)

set

In [48]:
chemistry_fos= list(childOfRoot["FOSChildID"]) + list(childAtLevel2)

In [49]:
type(chemistry_fos)

list

In [51]:
len(chemistry_fos)

136

## Physics 121332964

In [53]:
childOfRoot = ParentChild_DF[ParentChild_DF["FOS_ID"] == 121332964]

In [54]:
len(childOfRoot)

42

In [55]:
#Children at level 2
childAtLevel2 = set()
for x in childOfRoot.index:
    for y in ParentChild_DF.index:
        #print(ChildFOS_df["FOS_ID"][y])
        #print("\n")
        #print(x)
        if ParentChild_DF["FOS_ID"][y] == childOfRoot["FOSChildID"][x]:
            childAtLevel2.add(ParentChild_DF["FOSChildID"][y])

In [56]:
len(childAtLevel2)

229

In [57]:
#Children at level 3
childAtLevel3 = set()
for x in childAtLevel2:
    for y in ParentChild_DF.index:
        if ParentChild_DF["FOS_ID"][y] == x:
            childAtLevel3.add(ParentChild_DF["FOSChildID"][y])

In [58]:
len(childAtLevel3)

0

In [59]:
physics_fos= list(childOfRoot["FOSChildID"]) + list(childAtLevel2)

In [60]:
type(FOS_DF)

pyspark.sql.dataframe.DataFrame

## Fetching chemistry papers

In [61]:
FOS_DF_pd = FOS_DF.toPandas()

In [62]:
type(FOS_DF_pd)

pandas.core.frame.DataFrame

In [63]:
FOS_chemistry = FOS_DF[FOS_DF["FOS_ID"].isin(chemistry_fos)]

In [64]:
type(FOS_chemistry)

pyspark.sql.dataframe.DataFrame

In [65]:
FOS_chemistry.count()

136

In [67]:
txt_file = sc.textFile("../Data/graph/2018-02-02/PaperFieldsOfStudy.txt")

In [68]:
temp_var = txt_file.map(lambda k: k.split("\t"))

In [69]:
paperFOS_DF = temp_var.toDF()

In [70]:
paperFOS_DF.show(2)

+----------+---------+-------------------+
|        _1|       _2|                 _3|
+----------+---------+-------------------+
|2514067917|204223013|0.43515353105522547|
|2514067917|530198007| 0.4671860155535934|
+----------+---------+-------------------+
only showing top 2 rows



In [71]:
paperFOS_DF = paperFOS_DF.withColumn("PaperID",paperFOS_DF["_1"].cast(LongType()))

In [72]:
paperFOS_DF = paperFOS_DF.withColumn("FOS_ID",paperFOS_DF["_2"].cast(LongType()))

In [73]:
paperFOS_DF = paperFOS_DF.withColumn("FOSProbability",paperFOS_DF["_3"].cast(DoubleType()))

In [74]:
paperFOS_DF.show(2)

+----------+---------+-------------------+----------+---------+------------------+
|        _1|       _2|                 _3|   PaperID|   FOS_ID|    FOSProbability|
+----------+---------+-------------------+----------+---------+------------------+
|2514067917|204223013|0.43515353105522547|2514067917|204223013|0.4351535310552255|
|2514067917|530198007| 0.4671860155535934|2514067917|530198007|0.4671860155535934|
+----------+---------+-------------------+----------+---------+------------------+
only showing top 2 rows



In [75]:
drop_cols=["_1","_2","_3","FOSProbability"]
paperFOS_DF = paperFOS_DF.drop(*drop_cols)

In [76]:
#paperFOS_DF.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('PaperFOS.csv',header = 'true')

In [77]:
paperFOS_DF.head()

Row(PaperID=2514067917, FOS_ID=204223013)

In [78]:
FOS_chemistry.count()

136

In [79]:
paperFOS_DF.count()

796470949

In [81]:
type(paperFOS_DF)

pyspark.sql.dataframe.DataFrame

In [82]:
FOS_chemistry.show(2)

+---------+-------+-----------------+----------------+---------------+-------------+----------------+
|   FOS_ID|FOSRank|FOSNormalizedName|  FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+---------+-------+-----------------+----------------+---------------+-------------+----------------+
|  5791170|  15815| food engineering|Food engineering|              2|         1267|            9186|
|171250308|   8516|   nanotechnology|  Nanotechnology|              1|       500390|         7520761|
+---------+-------+-----------------+----------------+---------------+-------------+----------------+
only showing top 2 rows



In [83]:
#import pyspark.sql.functions as F

In [84]:
#papfos_filter = paperFOS_DF.filter(F.col("FOSID").isin(FOS_chemistry))

In [85]:
#papfos_filter = paperFOS_DF.filter(col("FOSID").isin(FOS_chemistry))

In [86]:
#fos_filter =FOS_DF.filter(F.col("FOSID").isin(FOS_chemistry))papfos_filter =

In [93]:
fos_filter = FOS_DF.join(paperFOS_DF, on=["FOS_ID"],how="inner")

In [94]:
fos_filter.columns

['FOS_ID',
 'FOSRank',
 'FOSNormalizedName',
 'FOSDisplayName',
 'FOSDisplayLevel',
 'FOSPaperCount',
 'FOSCitationCount',
 'PaperID']

In [96]:
fos_filter.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('PaperIDFOS.csv',header = 'true')

In [114]:
fos_filter.show(2)

+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+-----------------+
| FOS_ID|FOSRank|   FOSNormalizedName|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|   PaperID|   FOSProbability|
+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+-----------------+
|3910719|  13240|isotopes of vanadium|Isotopes of vanadium|              3|           72|             381|1963927026|0.561190523793405|
|3910719|  13240|isotopes of vanadium|Isotopes of vanadium|              3|           72|             381|2523956911|0.567940364509597|
+-------+-------+--------------------+--------------------+---------------+-------------+----------------+----------+-----------------+
only showing top 2 rows



In [97]:
drop_cols = ["FOSRank","FOSNormalizedName","FOSDisplayName","FOSDisplayLevel","FOSPaperCount","FOSCitationCount"]
fos_filter = fos_filter.drop(*drop_cols)

In [98]:
fos_filter.columns

['FOS_ID', 'PaperID']

In [99]:

#fos_filter.count()

In [100]:
fos_chem_filter = fos_filter.join(FOS_chemistry, on=["FOS_ID"],how="inner")

In [101]:
#fos_chem_filter.count()

In [102]:
fos_chem_filter.show(2)

+---------+----------+-------+------------------+------------------+---------------+-------------+----------------+
|   FOS_ID|   PaperID|FOSRank| FOSNormalizedName|    FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+---------+----------+-------+------------------+------------------+---------------+-------------+----------------+
|159985019|2347208822|   8727|composite material|Composite material|              1|      4821505|        22420136|
|159985019|2550523273|   8727|composite material|Composite material|              1|      4821505|        22420136|
+---------+----------+-------+------------------+------------------+---------------+-------------+----------------+
only showing top 2 rows



In [103]:
fos_chem_filter.columns

['FOS_ID',
 'PaperID',
 'FOSRank',
 'FOSNormalizedName',
 'FOSDisplayName',
 'FOSDisplayLevel',
 'FOSPaperCount',
 'FOSCitationCount']

In [104]:
paper_df = spark.read.options(inferSchema=True,header=True).csv("NewPaper.csv")

In [105]:
paper_df.show(2)

+----------+-----+----+-------+--------------------+---------+----------+---------+----------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+
|   PaperID| Rank| DOI|DocType|          PaperTitle|PaperYear| PaperDate|Publisher| JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|
+----------+-----+----+-------+--------------------+---------+----------+---------+----------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+
|1819117828|24472|null|   null|the distinctive f...|     2001|2001-04-01|     null|2596664605|        null|          null|  null| null|     null|    null|             2|            0|             0|
|2771061228|24172|null|   null|compresseur a vis...|     2006|2006-12-04|     null|      null|        null|          null|  null| null|     null|    null|             0|            0|             0|
+----

In [106]:
paper_df.select("DocType").distinct().show()

+-----------+
|    DocType|
+-----------+
|     Patent|
|       null|
|    Journal|
| Conference|
|BookChapter|
|       Book|
+-----------+



In [108]:
paper_df.filter((paper_df["DocType"] == "") | (paper_df["DocType"].isNotNull()) | (paper_df["DocType"]=="null")).count()

76252223

In [109]:
paper_df.count()

171227624

In [110]:
paper_df.columns

['PaperID',
 'Rank',
 'DOI',
 'DocType',
 'PaperTitle',
 'PaperYear',
 'PaperDate',
 'Publisher',
 'JournalID',
 'ConfSeriesID',
 'ConfInstanceID',
 'Volume',
 'Issue',
 'FirstPage',
 'LastPage',
 'ReferenceCount',
 'CitationCount',
 'EstimatedCount']

In [111]:
drop_cols = ["PaperTitle","OriginalTitle","DOI","PaperDate"]
paper_df = paper_df.drop(*drop_cols)

In [118]:
fos_chem_filterfinal = fos_chem_filter.join(paper_df, on=["PaperID"],how="inner")

In [119]:
fos_chem_filterfinal.show(2)

+-------+---------+-------+--------------------+--------------------+---------------+-------------+----------------+-----+-------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+
|PaperID|   FOS_ID|FOSRank|   FOSNormalizedName|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount| Rank|DocType|PaperYear|Publisher|JournalID|ConfSeriesID|ConfInstanceID|Volume|Issue|FirstPage|LastPage|ReferenceCount|CitationCount|EstimatedCount|
+-------+---------+-------+--------------------+--------------------+---------------+-------------+----------------+-----+-------+---------+---------+---------+------------+--------------+------+-----+---------+--------+--------------+-------------+--------------+
| 142535|153642686|   8764|infrared spectros...|Infrared spectros...|              2|       127513|         1735654|28587|   null|     1966|     null|     null|        null|          null|  null| null|    

In [121]:
fos_chem_filterfinal.count()

37302514

In [120]:
fos_filter.count()

796470949

In [130]:
fos_filter.show(2)

+----------+-----+--------------------+----------+---------+----------+---------+---------+------------+--------------+--------------+-------------+--------------+---------+-------+--------------------+---------------+-------------+----------------+-------+--------------------+--------------------+---------------+-------------+----------------+
|   PaperID| Rank|                 DOI|   DocType|PaperYear| PaperDate|Publisher|JournalID|ConfSeriesID|ConfInstanceID|ReferenceCount|CitationCount|EstimatedCount|   FOS_ID|FOSRank|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|FOSRank|   FOSNormalizedName|      FOSDisplayName|FOSDisplayLevel|FOSPaperCount|FOSCitationCount|
+----------+-----+--------------------+----------+---------+----------+---------+---------+------------+--------------+--------------+-------------+--------------+---------+-------+--------------------+---------------+-------------+----------------+-------+--------------------+--------------------+-------

In [122]:
fos_chem_filterfinal.coalesce(1).write.format('com.databricks.spark.csv').mode("overwrite").save('ChemistryData.csv',header = 'true')